In [2]:
import pandas as pd

In [5]:
marketing_campaign=pd.read_csv('marketing_campaign.csv', sep=',')

In [6]:
subscribers=pd.read_csv('subscribers.csv', sep=',')

In [7]:
users=pd.read_csv('users.csv', sep=',')

In [8]:
marketing_campaign.head()

,user_id,date_served,marketing_channel,variant,language_displayed,converted
0,a1000,1/1/18,House Ads,personalization,English,True
1,a1001,1/1/18,House Ads,personalization,English,True
2,a1002,1/1/18,House Ads,personalization,English,True
3,a1003,1/1/18,House Ads,personalization,English,True
4,a1004,1/1/18,House Ads,personalization,English,True


In [9]:
subscribers.head()

,user_id,subscribing_channel,date_subscribed,date_canceled,is_retained
0,a1000,House Ads,1/1/18,NaN,True
1,a1001,House Ads,1/1/18,NaN,True
2,a1002,House Ads,1/1/18,NaN,True
3,a1003,House Ads,1/1/18,NaN,True
4,a1004,House Ads,1/1/18,NaN,True


In [10]:
users.head()

,user_id,age_group,language_preferred
0,a1000,0-18 years,English
1,a1001,19-24 years,English
2,a1002,24-30 years,English
3,a1003,30-36 years,English
4,a1004,36-45 years,English


## Кол-во строк и типы

In [14]:
marketing_campaign.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10037 entries, 0 to 10036
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   user_id             10037 non-null  object
 1   date_served         10021 non-null  object
 2   marketing_channel   10022 non-null  object
 3   variant             10037 non-null  object
 4   language_displayed  10037 non-null  object
 5   converted           10022 non-null  object
dtypes: object(6)
memory usage: 470.6+ KB


In [15]:
subscribers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10037 entries, 0 to 10036
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   user_id              10037 non-null  object
 1   subscribing_channel  1856 non-null   object
 2   date_subscribed      1856 non-null   object
 3   date_canceled        577 non-null    object
 4   is_retained          1856 non-null   object
dtypes: object(5)
memory usage: 392.2+ KB


In [16]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10037 entries, 0 to 10036
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   user_id             10037 non-null  object
 1   age_group           10037 non-null  object
 2   language_preferred  10037 non-null  object
dtypes: object(3)
memory usage: 235.4+ KB


## Объединенный df

In [17]:
df = marketing_campaign.merge(users, how = 'outer', on = 'user_id')

In [18]:
df = df.merge(subscribers, how = 'outer', on = 'user_id' )

In [20]:
df.describe()

,user_id,date_served,marketing_channel,variant,language_displayed,converted,age_group,language_preferred,subscribing_channel,date_subscribed,date_canceled,is_retained
count,10037,10021,10022,10037,10037,10022,10037,10037,1856,1856,577,1856
unique,10037,31,5,2,4,2,7,4,5,31,115,2
top,a1000,1/15/18,House Ads,control,English,False,19-24 years,English,Instagram,1/16/18,4/2/18,True
freq,1,789,4733,5091,9793,8946,1682,9275,600,163,15,1279


In [64]:
df_c = df[df['converted']==True].groupby('subscribing_channel', as_index=False)['user_id'].count()  
df_n_c = df.groupby('marketing_channel', as_index=False)['user_id'].count() 

In [69]:
df_t=df_c.merge(df_n_c, left_on='subscribing_channel', right_on = 'marketing_channel', how='outer')

In [71]:
df_t['eff'] = df_t.user_id_x/df_t.user_id_y

In [72]:
#эффективность кампаний
df_t.sort_values('eff',ascending=False)

,subscribing_channel,user_id_x,marketing_channel,user_id_y,eff
0,Email,167,Email,565,0.295575
3,Instagram,265,Instagram,1871,0.141635
1,Facebook,237,Facebook,1860,0.127419
4,Push,83,Push,993,0.083585
2,House Ads,298,House Ads,4733,0.062962


In [74]:
#Определить количество игроков в каждой возрастной группе
df.groupby('age_group')['user_id'].count()

age_group
0-18 years     1539
19-24 years    1682
24-30 years    1568
30-36 years    1355
36-45 years    1353
45-55 years    1353
55+ years      1187
Name: user_id, dtype: int64

In [80]:
df.date_subscribed=pd.to_datetime(df.date_subscribed)

In [84]:
#Определить самую раннюю дату подписки на сервис
df.date_subscribed.min()

Timestamp('2018-01-01 00:00:00')

In [91]:
#Определить портрет аудитории удержанных подписчиков (по возрасту и языку)
df[df['is_retained']==True].groupby(['age_group', 'language_preferred'])['user_id'].count().sort_values(ascending=False)

age_group    language_preferred
19-24 years  English               304
24-30 years  English               250
0-18 years   English               210
30-36 years  English               114
45-55 years  English               108
36-45 years  English               107
55+ years    English                86
24-30 years  German                 15
19-24 years  German                 11
             Spanish                10
0-18 years   German                 10
30-36 years  Spanish                 7
45-55 years  Spanish                 6
0-18 years   Spanish                 6
36-45 years  Spanish                 5
24-30 years  Spanish                 5
55+ years    Spanish                 5
45-55 years  German                  3
0-18 years   Arabic                  3
24-30 years  Arabic                  3
45-55 years  Arabic                  2
36-45 years  German                  2
55+ years    Arabic                  2
             German                  2
36-45 years  Arabic             